<a href="https://www.kaggle.com/code/samithsachidanandan/cafa-6-protein-function-prediction-tf?scriptVersionId=268801947" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### IMPORTING LIBRARIES

In [122]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import os


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split

In [123]:
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

TensorFlow version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


### SETUP & CONFIGURATION

In [124]:
class config:
    MAIN_DIR = "/kaggle/input/cafa-6-protein-function-prediction"
    
 
    num_labels = 500
    n_epochs = 8
    batch_size = 64
    lr = 1e-3  # 0.001, 1e-4, 5e-4, 5e-3 tried  
    
    device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'

print(f"Using device: {config.device}")


embeds_map = {
    "T5": "t5embeds",
    "ProtBERT": "protbert-embeddings-for-cafa5",
    "EMS2": "cafa-5-ems-2-embeddings-numpy"
}
embeds_dim = {
    "T5": 1024,
    "ProtBERT": 1024,
    "EMS2": 1280
}

Using device: /GPU:0


### Loading the Data

In [125]:
def load_protein_data(datatype, embeddings_source):
 
    base_path = f"/kaggle/input/{embeds_map[embeddings_source]}/"
    
  
    embeds_path = os.path.join(base_path, f"{datatype}_embeddings.npy")
    ids_path = os.path.join(base_path, f"{datatype}_ids.npy")
    

    if embeddings_source == "T5":
        embeds_path = os.path.join(base_path, f"{datatype}_embeds.npy")

    embeds = np.load(embeds_path)
    ids = np.load(ids_path)
    
    if datatype == "train":
        labels_path = f"/kaggle/input/train-targets-top{config.num_labels}/train_targets_top{config.num_labels}.npy"
        labels = np.load(labels_path)
        return embeds, labels, ids
    else:
        return embeds, ids

### MODEL ARCHITECTURE: 1D CNN 

we are building a 1D Convolutional Neural Network (CNN) for multi-label classification. Staring with input layer that reshapes the data so that is it fitted as per the NN requirements then we are applying 32 filters to get the baic features then 3 more Conv1D are applied to get the advances features. We are using GlobalAveragePooling layer so that the features are reduces to a compact form. Followed by dense layer and drop out to reduce overfitting. 

In [126]:
def build_cnn_model(input_dim, num_classes):
    model = models.Sequential()
    
    
    model.add(layers.Input(shape=(input_dim,)))
    model.add(layers.Reshape((input_dim, 1)))
    

    model.add(layers.Conv1D(32, kernel_size=7, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    
 
    for filters in [64, 128, 256]:
        model.add(layers.Conv1D(filters, kernel_size=9, padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.Activation('relu'))
        model.add(layers.MaxPooling1D(pool_size=2))
    

    model.add(layers.GlobalAveragePooling1D())
    
  
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(num_classes, activation='sigmoid')) 
    
    return model

### CUSTOM METRICS

F1-score is calculatedby tracking true positives, false positives, and false negatives during training. Predictions are converted to binary values using a defined threshold (default 0.5). From these values, precision and recall are computed, and the F1-score is derived.  The metric is reset after each epoch in order to track the metric correctly. The metric has configuration methods to ensure it is fully serializable and can be saved and loaded with the model. Keras will automatically serialize and deserialize this metric when the model is trained and reloaded, because the class is decorated with the @keras.utils.register_keras_serializable decorator.

In [127]:
@keras.utils.register_keras_serializable(package="Custom", name="MultilabelF1Score")
class MultilabelF1Score(keras.metrics.Metric):
    
    
    def __init__(self, num_labels=500, threshold=0.5, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_labels = num_labels
        self.threshold = threshold
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred > self.threshold, tf.float32)
        y_true = tf.cast(y_true, tf.float32)
        
        tp = tf.reduce_sum(y_true * y_pred)
        fp = tf.reduce_sum((1 - y_true) * y_pred)
        fn = tf.reduce_sum(y_true * (1 - y_pred))
        
        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)
    
    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
        return f1
    
    def reset_state(self):
        self.true_positives.assign(0)
        self.false_positives.assign(0)
        self.false_negatives.assign(0)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'num_labels': self.num_labels,
            'threshold': self.threshold
        })
        return config
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

### THRESHOLD FINDING FUNCTION

A function is created for the optimal prediction threshold that gives the highest F1-score

In [128]:
def find_best_threshold(model, X_val, y_val, thresholds=np.arange(0.1, 0.51, 0.05)):
    
    predictions = model.predict(X_val, batch_size=config.batch_size, verbose=0)
    
    best_f1 = 0
    best_thresh = 0.5
    
    for thresh in thresholds:
        y_pred_binary = (predictions > thresh).astype(np.float32)
        
    
        tp = np.sum(y_val * y_pred_binary)
        fp = np.sum((1 - y_val) * y_pred_binary)
        fn = np.sum(y_val * (1 - y_pred_binary))
        
        precision = tp / (tp + fp + 1e-7)
        recall = tp / (tp + fn + 1e-7)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-7)
        
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = thresh
    
    return best_f1, best_thresh

### TRAINING FUNCTION

The function trains a convolutional neural network using the specified embedding source. It monitors performance on a validation set, calculates F1-score across multiple thresholds, and saves the model with the best F1 automatically. Finally, it returns the best model and optimal threshold for predictions.

In [129]:
# early stopping with 40 epochs
# def train_model(embeddings_source, model_type="convolutional", train_size=0.9):
#     print("Loading training data...")
#     X_train_full, y_train_full, ids = load_protein_data("train", embeddings_source)

#     X_train, X_val, y_train, y_val = train_test_split(
#         X_train_full, y_train_full, 
#         train_size=train_size, 
#         random_state=42
#     )

#     print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")

#     if model_type == "convolutional":
#         model = build_cnn_model(
#             input_dim=embeds_dim[embeddings_source], 
#             num_classes=config.num_labels
#         )
#     else:
#         raise ValueError("Unsupported model type")

#     model.compile(
#         optimizer=keras.optimizers.Adam(learning_rate=config.lr),
#         loss='binary_crossentropy',
#         metrics=[MultilabelF1Score(num_labels=config.num_labels)]
#     )

#     print(model.summary())

  
#     checkpoint = ModelCheckpoint(
#         'best_model.keras',
#         monitor='val_f1_score',
#         save_best_only=True,
#         mode='max',
#         verbose=1
#     )

#     reduce_lr = ReduceLROnPlateau(
#         monitor='val_f1_score',
#         factor=0.3,
#         patience=2,
#         mode='max',
#         verbose=1
#     )

#     early_stop = EarlyStopping(
#         monitor='val_f1_score',
#         patience=4,
#         mode='max',
#         restore_best_weights=True,
#         verbose=1
#     )

#     print("STARTING TRAINING...")

#     history = model.fit(
#         X_train, y_train,
#         batch_size=config.batch_size,
#         epochs=config.n_epochs,
#         validation_data=(X_val, y_val),
#         callbacks=[checkpoint, reduce_lr, early_stop],
#         verbose=1
#     )

  
#     model = keras.models.load_model('best_model.keras', custom_objects={"MultilabelF1Score": MultilabelF1Score})

  
#     best_val_f1, best_threshold = find_best_threshold(model, X_val, y_val)
#     print("\nTRAINING FINISHED")
#     print(f"Best Validation F1: {best_val_f1:.4f} at threshold {best_threshold:.2f}")

#     return model, best_threshold
# ems2_model, best_threshold = train_model(embeddings_source="EMS2", model_type="convolutional")

In [ ]:
def train_model(embeddings_source, model_type="convolutional", train_size=0.9):
    
    
    print("Loading training data...")
    X_train_full, y_train_full, ids = load_protein_data("train", embeddings_source)
    
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full, 
        train_size=train_size, 
        random_state=42
    )
    
    print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")
    
   
    if model_type == "convolutional":
        model = build_cnn_model(
            input_dim=embeds_dim[embeddings_source], 
            num_classes=config.num_labels
        )
    else:
        raise ValueError("Unsupported model type")
    
 
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=config.lr),
        loss='binary_crossentropy',
        metrics=[MultilabelF1Score(num_labels=config.num_labels)]
    )
    
    print(model.summary())
    
   
    checkpoint = ModelCheckpoint(
        'best_model.keras',
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=1
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_f1_score',
        factor=0.1,
        patience=1,
        mode='max',
        verbose=1
    )
    
    print("STARTING TRAINING...")
    
    
    best_val_f1 = 0.0
    best_threshold = 0.5
    
    for epoch in range(config.n_epochs):
        print(f"\nEPOCH {epoch+1}/{config.n_epochs}")
        
   
        history = model.fit(
            X_train, y_train,
            batch_size=config.batch_size,
            epochs=1,
            verbose=1,
            validation_data=(X_val, y_val)
        )
        
     
        val_f1, val_threshold = find_best_threshold(model, X_val, y_val)
        
        print(f"Validation F1-Score: {val_f1:.4f} (at threshold: {val_threshold:.2f})")
        
      
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            best_threshold = val_threshold
            model.save('best_model.keras')
            print(f"New best model saved! F1: {best_val_f1:.4f}")
        
        
        current_lr = float(model.optimizer.learning_rate.numpy())
        if epoch > 0 and val_f1 < best_val_f1:
            new_lr = current_lr * 0.1
            model.optimizer.learning_rate.assign(new_lr)
            print(f"Reducing learning rate to {new_lr}")
    
    print("\nTRAINING FINISHED")
    print(f"Highest Validation F1-Score: {best_val_f1:.4f}")
    print(f"Best threshold for this score: {best_threshold:.2f}")
    

    model = keras.models.load_model('best_model.keras')
    
    return model, best_threshold


ems2_model, best_threshold = train_model(embeddings_source="EMS2", model_type="convolutional")

Loading training data...
Training samples: 128021, Validation samples: 14225


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_11 (Reshape)            │ (None, 1280, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_46 (Conv1D)              │ (None, 1280, 32)       │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_46          │ (None, 1280, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_46 (Activation)      │ (None, 1280, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_46 (MaxPooling1D) │ (None, 640, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_47 (Conv1D)              │ (None, 640, 64)        │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_47          │ (None, 640, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_47 (Activation)      │ (None, 640, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_47 (MaxPooling1D) │ (None, 320, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_48 (Conv1D)              │ (None, 320, 128)       │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_48          │ (None, 320, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_48 (Activation)      │ (None, 320, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_48 (MaxPooling1D) │ (None, 160, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_49 (Conv1D)              │ (None, 160, 256)       │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_49          │ (None, 160, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_49 (Activation)      │ (None, 160, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_49 (MaxPooling1D) │ (None, 80, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_11     │ (None, 256)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 500)            │       512,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,165,364 (4.45 MB)

 Trainable params: 1,164,404 (4.44 MB)

 Non-trainable params: 960 (3.75 KB)

None
STARTING TRAINING...

EPOCH 1/8
2001/2001 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - f1_score: 0.1818 - loss: 0.1744 - val_f1_score: 0.1960 - val_loss: 0.1650
Validation F1-Score: 0.3447 (at threshold: 0.15)
New best model saved! F1: 0.3447

EPOCH 2/8
2001/2001 ━━━━━━━━━━━━━━━━━━━━ 29s 14ms/step - f1_score: 0.1833 - loss: 0.1653 - val_f1_score: 0.1960 - val_loss: 0.1652
Validation F1-Score: 0.3457 (at threshold: 0.20)
New best model saved! F1: 0.3457

EPOCH 3/8
2001/2001 ━━━━━━━━━━━━━━━━━━━━ 29s 14ms/step - f1_score: 0.1823 - loss: 0.1653 - val_f1_score: 0.1960 - val_loss: 0.1647
Validation F1-Score: 0.3446 (at threshold: 0.20)
Reducing learning rate to 0.00010000000474974513

EPOCH 4/8
2001/2001 ━━━━━━━━━━━━━━━━━━━━ 29s 14ms/step - f1_score: 0.1848 - loss: 0.1644 - val_f1_score: 0.1690 - val_loss: 0.1642
Validation F1-Score: 0.3448 (at threshold: 0.15)
Reducing learning rate to 1.0000000474974514e-05

EPOCH 5/8
2001/2001 ━━━━━━━━━━━━━━━━━━━━ 29s 14ms/step - f1_score: 0.1785 - loss: 0.1

### GENERATING PREDICTIONS 

In [ ]:
def predict(model, embeddings_source, threshold):
    
    
    print("\nLoading test data...")
    X_test, test_ids = load_protein_data("test", embeddings_source)
    
  
    labels_df = pd.read_csv(os.path.join(config.MAIN_DIR, "Train/train_terms.tsv"), sep="\t")
    top_terms = labels_df.groupby("term")["EntryID"].count().sort_values(ascending=False)
    labels_names = top_terms.head(config.num_labels).index.values
    
    print("GENERATING PREDICTIONS FOR THE TEST SET...")
    
    
    predictions = model.predict(X_test, batch_size=config.batch_size, verbose=1)
    
   
    results = []
    for i, protein_id in enumerate(tqdm(test_ids, desc="Processing predictions")):
        protein_probs = predictions[i]
        go_indices = np.where(protein_probs > threshold)[0]
        for idx in go_indices:
            results.append({
                "Id": protein_id,
                "GO term": labels_names[idx],
                "Confidence": protein_probs[idx]
            })
    
    submission_df = pd.DataFrame(results)
    print("PREDICTIONS COMPLETE.")
    return submission_df

submission_df = predict(ems2_model, "EMS2", best_threshold)



### SUBMISSION FILE GENERATION 

In [ ]:
print("\nMerging submission files...")


submission2 = pd.read_csv('/kaggle/input/blast-quick-sprof-zero-pred/submission.tsv',
                          sep='\t', header=None, names=['Id', 'GO term', 'Confidence2'])


subs = pd.merge(submission_df, submission2, on=['Id', 'GO term'], how='outer')


subs['Confidence_combined'] = subs['Confidence2'].fillna(subs['Confidence'])


final_submission = subs[['Id', 'GO term', 'Confidence_combined']]
final_submission.to_csv('submission.tsv', sep='\t', header=False, index=False)

print("Submission file 'submission.tsv' created successfully!")
print(f"It contains {len(final_submission)} predictions in total.")

Acknowledgement: - [https://www.kaggle.com/code/momerer/cafa-6-protein-function-prediction-with-1d-cnn](https://www.kaggle.com/code/momerer/cafa-6-protein-function-prediction-with-1d-cnn)